In [ ]:
import pandas as pd
import pgeocode
# 1. What is the number of distinct specialties for each territory?

# Load the hcp and ziptoter sheets from the Excel file
hcp = pd.read_excel('data/Data_Engineering.xlsx', sheet_name='hcp')
hco = pd.read_excel('data/Data_Engineering.xlsx', sheet_name='hco')
ziptoter = pd.read_excel('data/Data_Engineering.xlsx', sheet_name='ziptoter')

hcp = hcp.rename(columns={'zip': 'zipcode'})

# Merge the hcp and ziptoter DataFrames on the zipcode column
merged_df = pd.merge(hcp, ziptoter, on='zipcode')

# Group the resulting DataFrame by territory and specialty, and count the number of distinct specialties for each territory
specialties_per_territory = merged_df.groupby('ter')['specialty'].nunique()

# Print the resulting DataFrame
print(specialties_per_territory)

ter
east    6
mid     6
west    4
Name: specialty, dtype: int64


In [2]:
# 2.  What is the straight-line distance(km) for each hcp with a RHEUMATOLOGY or
# NEPHROLOGY specialty to the closest IC (infusion center) in their territory?

# Select all rows where specialty is RHEUMATOLOGY or NEPHROLOGY
selected_specialty = merged_df.loc[merged_df['specialty'].isin(['RHEUMATOLOGY', 'NEPHROLOGY'])]

# Print the selected rows
print("Specialties\n",selected_specialty.head())

merged_type = pd.merge(hco, ziptoter, on='zipcode')

#select all account_types from hco be IC
selected_type = merged_type.loc[merged_type['account_type'].isin(['IC'])]
print("Selected Type\n",selected_type.head())

dist = pgeocode.GeoDistance('us')
#dist.query_postal_code("60076", "60614")
# Calculate the distances between each HCP and IC in their territory
distances = []
for i, row in selected_specialty.iterrows():
    ic_df = selected_type.loc[selected_type['ter'] == row['ter']]
    for j, ic_row in ic_df.iterrows():
        distance_km = dist.query_postal_code(row['zipcode'], ic_row['zipcode'])
        distances.append({'npi': row['npi'], 'account_id': ic_row['account_id'], 'distance_km': distance_km})

# Convert the distances list to a DataFrame
distances_df = pd.DataFrame(distances)

# Group the resulting DataFrame by HCP and find the minimum distance for each HCP
min_distances = distances_df.groupby('npi')['distance_km'].min()

# Print the resulting DataFrame
print(min_distances.head())
min_distances

Specialties
          npi     specialty  zipcode   ter
0   85299899  RHEUMATOLOGY     1089  east
6   99076316    NEPHROLOGY     1915  east
7   20215085  RHEUMATOLOGY     2127  east
9   63125103  RHEUMATOLOGY     2860  east
13  62119406  RHEUMATOLOGY     3301  east
Selected Type
     account_id account_type  zipcode   ter
7       127672           IC     2864  east
8       127672           IC     2864  east
16      414858           IC     6410  east
26      273232           IC     8742  east
27      840809           IC     8859  east
npi
13752206    552.466979
15389047    331.600055
15504670    575.748574
16032534     71.900471
17069414    643.947149
Name: distance_km, dtype: float64


npi
13752206    552.466979
15389047    331.600055
15504670    575.748574
16032534     71.900471
17069414    643.947149
               ...    
97289547           NaN
98537554    330.358554
99076316           NaN
99550036    245.603821
99914729    164.135866
Name: distance_km, Length: 99, dtype: float64

In [3]:
# 3.  For each IC, what are the closest 3 hcps with a RHEUMATOLOGY or NEPHROLOGY
# specialty in the ICs territory?
top_distances = []
for i, row in selected_type.iterrows():
    ic_df = selected_specialty.loc[selected_specialty['ter'] == row['ter']]
    for j, ic_row in ic_df.iterrows():
        top_distance_km = dist.query_postal_code(row['zipcode'], ic_row['zipcode'])
        top_distances.append({'account_id': row['account_id'], 'npi': ic_row['npi'], 'distance_km': top_distance_km})

# Convert the distances list to a DataFrame
distances_df = pd.DataFrame(top_distances).dropna()

# Group the resulting DataFrame by HCP and find the minimum distance for each HCP
min_distances = distances_df.groupby('account_id')['distance_km'].nsmallest(3)
min_distances_df = pd.DataFrame(min_distances)

# Print the resulting DataFrame
print(min_distances_df.head(18))


min_distances_df
  


                distance_km
account_id                 
103616     582     8.262928
           575    18.447299
           574    38.035312
158543     942    40.391134
           940   161.064543
           941   161.064543
277320     462    16.434807
           455    23.964484
           454    27.689354
307867     870    22.916810
           925    22.916810
           871    48.241739
313116     930   247.875076
           931   247.875076
           932   406.342088
393923     323    69.057841
           324    69.057841
           322    70.959797


distance_km
account_id                  
103616     582      8.262928
           575     18.447299
           574     38.035312
158543     942     40.391134
           940    161.064543
           941    161.064543
277320     462     16.434807
           455     23.964484
           454     27.689354
307867     870     22.916810
           925     22.916810
           871     48.241739
313116     930    247.875076
           931    247.875076
           932    406.342088
393923     323     69.057841
           324     69.057841
           322     70.959797
633829     671     17.663296
           670    151.219025
           672    213.297721
644033     1052    77.110191
           1043   245.603821
           1041   249.910969
646281     937     26.531139
           935    161.591124
           936    161.591124
690805     747    107.187969
           757    184.387621
           755    189.343893
696208     1058   659.897966
           1057  2425.915571
           1055  2569.131554
718117     519     15.302719
           521     40.648350
           515     45.993149
800819     613     20.711092
           614     20.711092
           634    319.817168
873456     386    279.618557
           388    301.799159
           400    357.160359
965138     997     49.841883
           995    159.762183
           996    170.568625
974611     811     42.854549
           813     55.823598
           810     71.900471